# 데이터 수집(박근혜, 문재인, 윤석열)

### 모듈 불러오기

In [11]:
#모듈 불러오기
import re
import time
import urllib
import urllib.request
import json
import time
from bs4 import BeautifulSoup
import html
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np

### 윤석열 기사 수집

In [17]:
# API id, secret
client_id = 
client_secret = 

# 네이버 뉴스 API
base_url = 'https://openapi.naver.com/v1/search/news.json'

# 검색 키워드 설정
queries = [
    '"尹대통령" | "윤 대통령" | "윤석열" +2022',
    '"尹대통령" | "윤 대통령" | "윤석열" +2023',
    '"尹대통령" | "윤 대통령" | "윤석열" +2024',
    '"尹대통령" | "윤 대통령" | "윤석열" +공약',
    '"尹대통령" | "윤 대통령" | "윤석열" +정책'
]
n_display = 100
sort = 'date'

In [18]:
# 네이버 API 설정
all_links = []
all_pubdates = []

# HTML 태그 제거 함수
def remove_tag(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# 여러 해에 걸쳐 뉴스 링크와 발행일 가져오기
for query in queries:#검색어 리스트(queries) 입력
    encQuery = urllib.parse.quote(query)
    for start in range(1, 1001, 100):
        url = f'{base_url}?query={encQuery}&display={n_display}&start={start}&sort={sort}'
        my_request = urllib.request.Request(url)
        my_request.add_header("X-Naver-Client-Id", client_id)
        my_request.add_header("X-Naver-Client-Secret", client_secret)
        
        try:
            response = urllib.request.urlopen(my_request)
            rescode = response.getcode()
            if rescode == 200:
                response_body = response.read()
            else:
                print("Error Code:" + rescode)
                continue
        except urllib.error.HTTPError as e:
            print(f"HTTP Error: {e.code} - {e.reason}")
            continue
        except urllib.error.URLError as e:
            print(f"URL Error: {e.reason}")
            continue

        search_results = response_body.decode('utf-8')
        search_results = json.loads(search_results)
        
        if 'items' not in search_results:
            print("No items in search results")
            continue

        p = re.compile('n.news.naver.com')# 링크가 네이버뉴스인 것만 수집
        for item in search_results['items']:
            if p.search(item['link']) is not None:
                title = html.unescape(remove_tag(item['title']))
                pubdate = item['pubDate']
                link = item['link'].replace('\\', '')
                all_links.append(link)
                all_pubdates.append(pubdate)
        
        time.sleep(1)  # API 호출 제한 방지를 위한 대기 시간

# 링크 및 날짜 데이터프레임 생성
df_link = pd.DataFrame({'link': all_links, 'pubdate': all_pubdates})

In [19]:
df_link

,link,pubdate
0,https://n.news.naver.com/mnews/article/079/000...,"Thu, 30 May 2024 22:45:00 +0900"
1,https://n.news.naver.com/mnews/article/018/000...,"Thu, 30 May 2024 17:38:00 +0900"
2,https://n.news.naver.com/mnews/article/079/000...,"Thu, 30 May 2024 05:11:00 +0900"
3,https://n.news.naver.com/mnews/article/022/000...,"Wed, 29 May 2024 18:19:00 +0900"
4,https://n.news.naver.com/mnews/article/079/000...,"Tue, 28 May 2024 23:06:00 +0900"
...,...,...
2860,https://n.news.naver.com/mnews/article/008/000...,"Sun, 28 Apr 2024 16:31:00 +0900"
2861,https://n.news.naver.com/mnews/article/421/000...,"Sun, 28 Apr 2024 15:30:00 +0900"
2862,https://n.news.naver.com/mnews/article/025/000...,"Sat, 27 Apr 2024 14:11:00 +0900"
2863,https://n.news.naver.com/mnews/article/009/000...,"Sat, 27 Apr 2024 09:23:00 +0900"


In [33]:
# Selenium을 사용하여 각 링크에서 데이터 수집
driver = webdriver.Chrome()
driver.implicitly_wait(3)

In [34]:
results = []

for i, url in enumerate(df_link['link']):
    driver.get(url)
    
    # 제목, 본문, 언론사 얻기
    #제목
    try:
        title = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="title_area"]/span'))
        ).text
    except Exception as e:
        print(f"Error fetching title for URL {url}: {e}")
        title = ""
        
    #본문
    try:
        content = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="newsct_article"]'))
        ).text
    except Exception as e:
        print(f"Error fetching content for URL {url}: {e}")
        content = ""
    
    #언론사
    try:
        copyright_text = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.XPATH, '//p[@class="c_text"]'))
        ).text
        if "ⓒ" in copyright_text:
            agency = copyright_text.split("ⓒ")[1].strip()
        else:
            agency = ""
    except Exception as e:
        print(f"Error fetching agency for URL {url}: {e}")
        agency = ""
    
    # 댓글 얻기
    try:
        all_comment_btn = WebDriverWait(driver, 2).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'u_cbox_in_view_comment'))
        )
        all_comment_btn.click()
        
        comments = []
        while True:
            try:
                more_comment_btn = WebDriverWait(driver, 2).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, 'u_cbox_btn_more'))
                )
                more_comment_btn.click()
                time.sleep(1)
            except:
                break
        
        comments_elements = driver.find_elements(By.CLASS_NAME, 'u_cbox_contents')
        comments = [x.text for x in comments_elements]
    except Exception as e:
        print(f"Error fetching comments for URL {url}: {e}")
        comments = []

    time.sleep(1)
    results.append([url, title, agency, content, comments]) # 결과 수집

driver.quit()

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012534490?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0012534444?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002872200?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/079/0003892556?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002872088?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002872038?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/087/0001042444?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/656/0000089166?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002871162?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002834639?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002833963?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/022/0003869866?sid=110: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/001/0014275436?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/001/0014274764?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/079/0003819766?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002828284?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002316650?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/037/0000033155?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/005/0001694962?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/660/0000060829?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/014/0005183338?sid=101: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002872732?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/016/0002306451?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/016/0002306434?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/016/0002306320?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/656/0000089575?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002872677?sid=100: Message: 

Error fetching comments for URL https://n.news

In [22]:
# 크롤링 데이터 프레임 생성(url, 기사 제목, 언론사, 댓글)
df_yoon = pd.DataFrame(results, columns=['url', 'title', 'agency', 'content', 'comment'])
df_yoon

,url,title,agency,content,comment
0,https://n.news.naver.com/mnews/article/079/000...,"尹, 與 워크숍 3년 연속 참석…""당과 한몸으로 뼈 빠지게 뛰겠다""",노컷뉴스. All rights reserved. 무단 전재 및 재배포 금지.,"尹대통령, 제22대 국민의힘 국회의원 워크숍 참석\n'당정 일체' 강조하며 ""나라를...","[🥵🥸🤯😎어휴 저 병원신 같은놈...귀신은 뭐하냐 저런거 안잡아가고..., 특검 막..."
1,https://n.news.naver.com/mnews/article/018/000...,尹대통령 “우주 개척시대 ‘스페이스 스탠다드’ 선도할 것”,이데일리. All rights reserved. 무단 전재 및 재배포 금지.,"우주항공청 개청식 참석…우주기술, 최첨단 기술 집합체\n달·화성 착륙 ‘스페이스 광...","[깝치기보단 내실을. 경제는 왜국에 파냐.?, 우주시대 선도 선언!( R&D 예산을..."
2,https://n.news.naver.com/mnews/article/079/000...,"尹, UAE 대통령과 '경제협력' 강화…'새로운 중동붐' 모멘텀",노컷뉴스. All rights reserved. 무단 전재 및 재배포 금지.,"尹대통령, 첫 방한 UAE 대통령과 정상회담\n아랍 국가 최초로 포괄적경제동반자협정...","[이번 계약서 전문 공개해라, 쥐박이 처럼 이면 계약으로 유사시 국군파병 조약 쳐넣..."
3,https://n.news.naver.com/mnews/article/022/000...,장관 사라진 여가부 여성정책도 사라졌다 [심층기획-여가부 장관 공백 사태 100일],세계일보. All rights reserved. 무단 전재 및 재배포 금지.,수장 없는 부처 ‘있으나 마나’… 표류하는 성평등·여성정책\n\n차관 직무대행 체제...,[여가부 20여년의 성과는 지금 출산율과 젠더 갈등으로 증명되지 않았나? 없어지는 ...
4,https://n.news.naver.com/mnews/article/079/000...,"尹대통령 부부, UAE 대통령과 친교 일정…창덕궁 산책 후 만찬",노컷뉴스. All rights reserved. 무단 전재 및 재배포 금지.,UAE 대통령 최초 국빈방문\n윤석열 대통령이 28일 서울 종로구 창덕궁 후원 부용...,"[가가멜과 스머프가 떠오르는 건 뭐지...., 정치 경재는 알고 국빈들하고 대화하는..."
...,...,...,...,...,...
1740,https://n.news.naver.com/mnews/article/016/000...,"“금투세 폐지, 국민 바라는 법안...반도체는 시간이 보조금” [尹대통령 취임 2년...",헤럴드경제. All rights reserved. 무단 전재 및 재배포 금지.,"관련 법안들 발의 야당의 협조 필요\n‘韓, 올 성장률 2.6% 상향 조정’ 역설\...",[금투세 폐지는 진작에 되어야했습니다 지금까지 끌고온것도 말이안돼요]
1741,https://n.news.naver.com/mnews/article/016/000...,尹 “ISA 비과세 한도 확대·금투세 폐지…경제 회복의 청신호” [尹대통령 취임 2년],헤럴드경제. All rights reserved. 무단 전재 및 재배포 금지.,관련 법안들 巨野 협조 필요\n尹 “경제 회복 청신호 들어와”\n“시장경제·건전재정...,"[금투세 폐지 반대가 민주당 전략인데, 폐지 하겠는가? 한국 경제가 망해야 대통령을..."
1742,https://n.news.naver.com/mnews/article/057/000...,"정부, 저출산고령사회위 정식 부처로 승격 추진","MBN. All rights reserved. 무단 전재, 재배포 및 AI학습 이용...","정부조직법 개정 필요…尹 대통령, 오늘 회견서 야당에 협조 구할 듯\n자료사진 / ...","[돈도 시간도 없다, 돈걱정 없이 애낳을수 있도록 신혼부부 대상 임대주택을 보급하고..."
1743,https://n.news.naver.com/mnews/article/001/001...,"정부, 저출산고령사회위 정식 부처로 승격 추진","연합뉴스. All rights reserved. 무단 전재-재배포, AI 학습 및 ...","정부조직법 개정 필요…尹대통령, 오늘 회견서 야당에 협조 구할듯\n여야 모두 지난 ...","[출생은 안늘고 혈세 까먹는 위원회 간부만 느는게지, 또 시간만 허비하겠구나.. 이..."


In [23]:
df_yoon.to_csv('news_data_윤3.csv', encoding = 'utf-8')

In [35]:
df_yoon2 = pd.DataFrame(results, columns=['url', 'title', 'agency', 'content', 'comment'])

In [36]:
df_yoon2

,url,title,agency,content,comment
0,https://n.news.naver.com/mnews/article/003/001...,"美전문가 ""트럼프 재집권땐 尹대통령과의 관계 더 중요해""",뉴시스. All rights reserved. 무단 전재 및 재배포 금지.,"""정책 전개 예측 어려워…굳건한 동맹 이익 고려해야""\n신원식 국방장관 ""강한 軍으...",[]
1,https://n.news.naver.com/mnews/article/003/001...,"尹대통령, 내일 기자회견…""대국민사과·특검법 수용하라""",뉴시스. All rights reserved. 무단 전재 및 재배포 금지.,"참여자치21, 성명내고 7개 이행사항 요구\n""민주주의 퇴행과 민생 파탄 대국민 사...",[]
2,https://n.news.naver.com/mnews/article/008/000...,"결국 '민정의 부활'…尹, 남은 3년 '민심잡기'에 사활",머니투데이. All rights reserved. 무단 전재 및 재배포 금지.,[the300]신임 민정수석에 김주현 전 대검 차장 인선\n또 웃으며 질문받은 尹대...,"[민정수석이 검사출신? 검찰공화국, 갈수록 태산이네., 얼굴에 수심이 가득하다. 아..."
3,https://n.news.naver.com/mnews/article/081/000...,脫청와대·가치 외교·우주항공청 공약 지켰지만 노동·교육·연금 등 3대 개혁 추진은 ...,서울신문. All rights reserved. 무단 전재 및 재배포 금지.,"尹정부 2년 ‘성과와 과제’\n‘국민이 키운 윤석열, 내일을 바꾸는 대통령’을 대선...",[배신하고 민주당 몰표 줘서 그렀지]
4,https://n.news.naver.com/mnews/article/658/000...,공약 파기 논란…尹 “민심 듣겠다” 野 “사정기관 장악 의도”,국제신문. All rights reserved. 무단 전재 및 재배포 금지.,"용산 민정수석실 부활 공방\n- 尹대통령, 국민 위한 개편 강조\n- “김대중 때도...",[이미 민심은 총선을 통해서 알텐데 무슨 꼼수를 부리려고 말에대한 책임감을 못느끼는...
...,...,...,...,...,...
1115,https://n.news.naver.com/mnews/article/008/000...,"첫 영수회담 'D-1'…이재명, 尹대통령 만나 무슨 얘기 나눌까",머니투데이. All rights reserved. 무단 전재 및 재배포 금지.,[the300]29일 尹정부 첫 영수회담\n(서울=뉴스1) 오대일 기자 = 윤석열 ...,"[kt 민영화 됐다..,충격....국민몰래 현대차에 팔았다....ytn민영화, tb..."
1116,https://n.news.naver.com/mnews/article/421/000...,"尹대통령-이재명, 첫 영수회담…이번주(29일~5월3일) 주요일정","뉴스1. All rights reserved. 무단 전재 및 재배포, AI학습 이용...",윤석열 대통령(왼쪽)과 이재명 더불어민주당 대표가 29일 용산 대통령실에서 첫 영수...,[]
1117,https://n.news.naver.com/mnews/article/025/000...,"국힘 ""文정부 대북정책 반성하라""…민주 ""尹대통령 강대강 대치 멈추라""",중앙일보. All rights reserved. 무단 전재 및 재배포 금지.,지난 2018년 4월 27일 문재인 대통령과 북한 김정은 국무위원장이 27일 판문점...,"[반성이고 뭐고 지발 앞 일만 생각 항션 좋겄는디, 멸공 국방력을 잘하고 있고만 ~..."
1118,https://n.news.naver.com/mnews/article/009/000...,“대통령이 책임져야” “영남당이라서 졌다”...여당 총선참패 진짜 원인은 [저격],"매일경제. All rights reserved. 무단 전재, 재배포 및 AI학습 이...",[저격-23] 4·10 총선 결과는 더불어민주당(야당)의 압승이었습니다.\n\n비례...,[윤석열이 말아먹은 선거 그나마 한동훈의 투혼으로 108석이라도 건졌다. 불통 대통...


In [37]:
df_yoon2.to_csv('news_data_윤4.csv', encoding = 'utf-8')

#### 끊어서 진행하여 concat
- news_data_윤5는 팀원이 수집

In [43]:
a= pd.read_csv('news_data_윤3.csv', encoding = 'utf-8')
b= pd.read_csv('news_data_윤4.csv', encoding = 'utf-8')
c= pd.read_csv('news_data_윤5.csv', encoding = 'utf-8')

In [44]:
data=pd.concat([a,b,c], ignore_index=True)

In [45]:
data= data[['url', 'title', 'agency', 'content', 'comment']]

In [46]:
data

,url,title,agency,content,comment
0,https://n.news.naver.com/mnews/article/079/000...,"尹, 與 워크숍 3년 연속 참석…""당과 한몸으로 뼈 빠지게 뛰겠다""",노컷뉴스. All rights reserved. 무단 전재 및 재배포 금지.,"尹대통령, 제22대 국민의힘 국회의원 워크숍 참석\n'당정 일체' 강조하며 ""나라를...","['🥵🥸🤯😎어휴 저 병원신 같은놈...귀신은 뭐하냐 저런거 안잡아가고...', '특..."
1,https://n.news.naver.com/mnews/article/018/000...,尹대통령 “우주 개척시대 ‘스페이스 스탠다드’ 선도할 것”,이데일리. All rights reserved. 무단 전재 및 재배포 금지.,"우주항공청 개청식 참석…우주기술, 최첨단 기술 집합체\n달·화성 착륙 ‘스페이스 광...","['깝치기보단 내실을. 경제는 왜국에 파냐.?', '우주시대 선도 선언!( R&D ..."
2,https://n.news.naver.com/mnews/article/079/000...,"尹, UAE 대통령과 '경제협력' 강화…'새로운 중동붐' 모멘텀",노컷뉴스. All rights reserved. 무단 전재 및 재배포 금지.,"尹대통령, 첫 방한 UAE 대통령과 정상회담\n아랍 국가 최초로 포괄적경제동반자협정...","['이번 계약서 전문 공개해라, 쥐박이 처럼 이면 계약으로 유사시 국군파병 조약 쳐..."
3,https://n.news.naver.com/mnews/article/022/000...,장관 사라진 여가부 여성정책도 사라졌다 [심층기획-여가부 장관 공백 사태 100일],세계일보. All rights reserved. 무단 전재 및 재배포 금지.,수장 없는 부처 ‘있으나 마나’… 표류하는 성평등·여성정책\n\n차관 직무대행 체제...,['여가부 20여년의 성과는 지금 출산율과 젠더 갈등으로 증명되지 않았나? 없어지는...
4,https://n.news.naver.com/mnews/article/079/000...,"尹대통령 부부, UAE 대통령과 친교 일정…창덕궁 산책 후 만찬",노컷뉴스. All rights reserved. 무단 전재 및 재배포 금지.,UAE 대통령 최초 국빈방문\n윤석열 대통령이 28일 서울 종로구 창덕궁 후원 부용...,"['가가멜과 스머프가 떠오르는 건 뭐지....', '정치 경재는 알고 국빈들하고 대..."
...,...,...,...,...,...
6098,https://n.news.naver.com/mnews/article/018/000...,"尹, 과거 수사했던 ‘朴최측근’ 정호성 비서관에 발탁",이데일리. All rights reserved. 무단 전재 및 재배포 금지.,시민사회수석실 3비서관 내정\n박근혜 정부 시절 ‘문고리 3인방’ 불려\n[이데일리...,['허거억 ㅋㅋㅋ 문고리 3인방 비서관 발탁해 ㅋㅋㅋ ㅋㅋㅋ 정신줄 놨구나 니들은 ...
6099,https://n.news.naver.com/mnews/article/421/000...,"尹 ""5000만에서 80억 시장으로 도전…기업하기 좋은 환경 만들 것""(종합2보)","뉴스1. All rights reserved. 무단 전재 및 재배포, AI학습 이용...",3년 연속 대통령실 잔디마당서 중소기업인 대회 개최\n이재용·구광모·장인화·김동관·...,['네이버 라인도 못지키는 술꾼이 또 취해서 주정부리네. 야당은 잔말말고 빨리 끌어...
6100,https://n.news.naver.com/mnews/article/119/000...,"윤 대통령, '朴 최측근' 정호성 비서관으로 발탁",데일리안. All rights reserved. 무단 전재 및 재배포 금지.,"尹, 서울중앙지검장 시절\n정호성 관련 사건 수사 지휘\n지난 2022년 특별사면하...","['이게 정상적인 사람인가요?', '비정상의 정상화가 시급함.', 'ㅎㅎ 국민을 바..."
6101,https://n.news.naver.com/mnews/article/366/000...,"尹대통령이 수사했던 ‘朴 최측근’ 정호성, 시민사회수석실 비서관 발탁",조선비즈. All rights reserved. 무단 전재 및 재배포 금지.,윤석열 대통령이 과거 박근혜 정부 시절 '문고리 3인방' 중 한 명으로 꼽혔던 정호...,['평생 검사질만 하던 놈을 뽑아놓으니 인재풀이라고 해봐야 지가 부려먹던 검사와 지...


#### 중복제거, 공백 제거, 언론사열 전처리

In [47]:
%%time
print('전처리 전 학습용 샘플의 개수 :',len(data))

# document 열에서 중복인 내용이 있다면 중복 제거
data.drop_duplicates(subset = ['content'], inplace=True) 

# train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
# 공백은 empty 값으로 변경
data['content'] = data['content'].str.replace('^ +', "")

# document column이 비어있다면(공백), Null 값으로 변경
data['content'].replace('', np.nan, inplace=True)

# Null 값 제거
data = data.dropna(how='any')

#언론사만 나오도록 바꾸기
data['agency'] = data['agency'].str.split('.').str[0]

# index reset
data = data.reset_index(drop=True)
print('전처리 후 학습용 샘플의 개수 :',len(data))

전처리 전 학습용 샘플의 개수 : 6103
전처리 후 학습용 샘플의 개수 : 3575
CPU times: total: 46.9 ms
Wall time: 41.5 ms


<timed exec>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [48]:
# 최종 크롤링 데이터 저장
data.to_csv('news_data_윤_final.csv', encoding = 'utf-8')

### 문재인 기사 수집

In [3]:
client_id = '
client_secret = 

base_url = 'https://openapi.naver.com/v1/search/news.json'
# 검색 키워드 설정
queries = [
    '"文대통령" | "문 대통령" | "문재인" +2017',
    '"文대통령" | "문 대통령" | "문재인" +2018',
    '"文대통령" | "문 대통령" | "문재인" +2019',
    '"文대통령" | "문 대통령" | "문재인" +2020',
    '"文대통령" | "문 대통령" | "문재인" +2021',
    '"文대통령" | "문 대통령" | "문재인" +2022'
]
n_display = 100
sort = 'sim'

In [4]:
# 네이버 API 설정
all_links = []
all_pubdates = []

# HTML 태그 제거 함수
def remove_tag(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# 여러 해에 걸쳐 뉴스 링크와 발행일 가져오기
for query in queries:
    encQuery = urllib.parse.quote(query)
    for start in range(1, 1001, 100):
        url = f'{base_url}?query={encQuery}&display={n_display}&start={start}&sort={sort}'
        my_request = urllib.request.Request(url)
        my_request.add_header("X-Naver-Client-Id", client_id)
        my_request.add_header("X-Naver-Client-Secret", client_secret)
        
        try:
            response = urllib.request.urlopen(my_request)
            rescode = response.getcode()
            if rescode == 200:
                response_body = response.read()
            else:
                print("Error Code:" + rescode)
                continue
        except urllib.error.HTTPError as e:
            print(f"HTTP Error: {e.code} - {e.reason}")
            continue
        except urllib.error.URLError as e:
            print(f"URL Error: {e.reason}")
            continue

        search_results = response_body.decode('utf-8')
        search_results = json.loads(search_results)
        
        if 'items' not in search_results:
            print("No items in search results")
            continue

        p = re.compile('n.news.naver.com')
        for item in search_results['items']:
            if p.search(item['link']) is not None:
                title = html.unescape(remove_tag(item['title']))
                pubdate = item['pubDate']
                link = item['link'].replace('\\', '')
                all_links.append(link)
                all_pubdates.append(pubdate)
        
        time.sleep(1)  # API 호출 제한 방지를 위한 대기 시간

df_link = pd.DataFrame({'link': all_links, 'pubdate': all_pubdates})

In [5]:
df_link

,link,pubdate
0,https://n.news.naver.com/mnews/article/081/000...,"Thu, 14 Dec 2023 15:55:00 +0900"
1,https://n.news.naver.com/mnews/article/421/000...,"Fri, 05 Jan 2024 07:31:00 +0900"
2,https://n.news.naver.com/mnews/article/366/000...,"Tue, 05 Apr 2022 16:31:00 +0900"
3,https://n.news.naver.com/mnews/article/003/001...,"Mon, 28 Mar 2022 20:02:00 +0900"
4,https://n.news.naver.com/mnews/article/366/000...,"Sat, 07 May 2022 14:11:00 +0900"
...,...,...
4326,https://n.news.naver.com/mnews/article/449/000...,"Mon, 25 Oct 2021 20:00:00 +0900"
4327,https://n.news.naver.com/mnews/article/366/000...,"Mon, 25 Oct 2021 18:11:00 +0900"
4328,https://n.news.naver.com/mnews/article/022/000...,"Mon, 25 Oct 2021 17:45:00 +0900"
4329,https://n.news.naver.com/mnews/article/088/000...,"Mon, 25 Oct 2021 16:59:00 +0900"


In [6]:
# Selenium을 사용하여 각 링크에서 데이터 수집
driver = webdriver.Chrome()
driver.implicitly_wait(3)

In [7]:
results = []

for i, url in enumerate(df_link['link']):
    driver.get(url)
    
    # 제목, 본문, 언론사 얻기
    try:
        title = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="title_area"]/span'))
        ).text
    except Exception as e:
        print(f"Error fetching title for URL {url}: {e}")
        title = ""

    try:
        content = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="newsct_article"]'))
        ).text
    except Exception as e:
        print(f"Error fetching content for URL {url}: {e}")
        content = ""

    try:
        copyright_text = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.XPATH, '//p[@class="c_text"]'))
        ).text
        if "ⓒ" in copyright_text:
            agency = copyright_text.split("ⓒ")[1].strip()
        else:
            agency = ""
    except Exception as e:
        print(f"Error fetching agency for URL {url}: {e}")
        agency = ""
    
    # 댓글 얻기
    try:
        all_comment_btn = WebDriverWait(driver, 1).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'u_cbox_in_view_comment'))
        )
        all_comment_btn.click()
        
        comments = []
        while True:
            try:
                more_comment_btn = WebDriverWait(driver, 1).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, 'u_cbox_btn_more'))
                )
                more_comment_btn.click()
                time.sleep(1)
            except:
                break
        
        comments_elements = driver.find_elements(By.CLASS_NAME, 'u_cbox_contents')
        comments = [x.text for x in comments_elements]
    except Exception as e:
        print(f"Error fetching comments for URL {url}: {e}")
        comments = []

    time.sleep(1)
    results.append([url, title, agency, content, comments])

driver.quit()

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002836206?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002238446?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002674941?sid=100: Message: 

Error fetching title for URL https://n.news.naver.com/mnews/article/008/0004583688?sid=100: Message: 

Error fetching content for URL https://n.news.naver.com/mnews/article/008/0004583688?sid=100: Message: 

Error fetching agency for URL https://n.news.naver.com/mnews/article/008/0004583688?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/008/0004583688?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/008/0004530289?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/629/0000047685?sid=100: Message: 

Error fetching comments for URL https://n.news.naver

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0002743798?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0002743397?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/011/0003035785?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/011/0003035758?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/011/0003034884?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0002737254?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/011/0003033380?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/011/0003032848?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0002733321?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/008/0004108870?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002582926?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/018/0004400571?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/119/0002374140?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0004373413?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/018/0004319742?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0004169404?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/018/0004056510?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/426/0000006728?sid=123: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/020/0003089006?sid=101: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/417/0000736938?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002672553?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002270842?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002272886?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/025/0002949537?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0006080045?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/005/0001524205?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002268256?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/022/0003369506?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002204891?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/018/0004141077?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/119/0002333177?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0003905535?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/016/0001476352?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/016/0001475555?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/023/0003415391?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/018/0004289199?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002265030?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002264909?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002264360?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/005/0001528853?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002262451?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002261738?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002254877?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002254428?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/005/0001521483?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002272886?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/025/0002949537?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/298/0000287446?sid=123: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/005/0001523972?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002274160?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002268256?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/016/0001980280?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002271427?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/656/0000007905?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/057/0001661186?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002254428?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/016/0001913683?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002730589?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002272744?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002259506?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002259327?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/005/0001519991?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002272222?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002260481?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/277/0005005319?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002244651?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/008/0004673293?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0005738184?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002260131?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002707218?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002260129?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/008/0004673278?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/005/0001513693?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002270101?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002269751?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/449/0000223878?sid=115: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/277/0005061786?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/030/0003002975?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/005/0001510367?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/123/0002254442?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/029/0002722777?sid=100: Message: 

Error fetching comments for URL https://n.news

In [8]:
df_moon = pd.DataFrame(results, columns=['url', 'title', 'agency', 'content', 'comment'])
df_moon

,url,title,agency,content,comment
0,https://n.news.naver.com/mnews/article/081/000...,"“文 대통령, 일본에 따박따박 대응하라고 했다” 청와대 비서관 회고",서울신문. All rights reserved. 무단 전재 및 재배포 금지.,문재인 전 대통령이 19일 오후 서울 영등포구 63빌딩에서 열린 9·19 평양공동선...,[중국몽 정신병자.. 왜 북한이나 중국에는 따박따박 대응 못하고 자국 기자가 공안에...
1,https://n.news.naver.com/mnews/article/421/000...,"文·尹·李 이어 한동훈도 ""5·18정신 헌법수록""…그런데 왜 아직까지?","뉴스1. All rights reserved. 무단 전재 및 재배포, AI학습 이용...","헌법 자문위 ""합당치 않다"" 의견… 文대통령 개헌안 '자동폐기'\n5·18관계자 ""...","[윤석열/한동훈 전두환/노태우 를 보는듯, 헌법은 커녕 망언이나 안하면 그나마 다행..."
2,https://n.news.naver.com/mnews/article/366/000...,‘김신조 사건’ 54년 만에 靑 뒤 북악산 전면 개방…文대통령 부부 산행,조선비즈. All rights reserved. 무단 전재 및 재배포 금지.,"靑 “文대통령, 대선 때 북악산 전면 개방 공약”\n“국민과의 약속 지킨 것”…20...","[여테있다가 새정부 개방한다닌깐ᆢ좀스럽다ᆢ좌파들, 니가 개방하냐, 숟가락 얻는데는 ..."
3,https://n.news.naver.com/mnews/article/003/001...,집무실 아래까지 마중 나온 文대통령…尹당선인 '극진 예우'(종합),뉴시스. All rights reserved. 무단 전재 및 재배포 금지.,기사내용 요약\n1년 9개월만에 대면한 文…엷은 미소로 尹에 악수\n상춘재까지 걸으...,[사치킹 필리핀 이멜다 징역 77년 선고ㅋㅋㅋㅋ K-이멜다 김돼숙도 징역 77년 가...
4,https://n.news.naver.com/mnews/article/366/000...,文대통령 “국회의원 적성에 맞나” 고민정 “열심히 싸우고 있다”,조선비즈. All rights reserved. 무단 전재 및 재배포 금지.,文대통령 “상대방에 의해 정의되지 말고 내가 왜 하는지를 찾아야지요”\n\n문재인 ...,"[생각없는 의원이군. 싸우려고 국회의원하나?, 북한 주체사상 영향 받은 애들 특징이..."
...,...,...,...,...,...
4326,https://n.news.naver.com/mnews/article/449/000...,文 대통령 “K방역·성장·선진국”…野 “고장난 라디오”,채널A. All rights reserved. 무단 전재 및 재배포 금지.,오늘 국회에선 문재인 대통령의 임기 마지막 시정연설이 있었습니다.\n\n문 대통령은...,[]
4327,https://n.news.naver.com/mnews/article/366/000...,野 반대하는 文대통령·이재명 만남…靑 “정치적 오해 없을 것”,조선비즈. All rights reserved. 무단 전재 및 재배포 금지.,"“2012년 이명박 대통령·박근혜 후보와 같은 유권해석”\n이재명, 전날 이낙연 면...",[대통령은 나라를 대표하는 사람입니다. 더 이상 특정 정당의 당원처럼 활동해선 안된...
4328,https://n.news.naver.com/mnews/article/022/000...,"文대통령, 고용의 질·남북관계 악화에도 “모두 이겨냈다”",세계일보. All rights reserved. 무단 전재 및 재배포 금지.,‘핵심’ 빠진 마지막 국회연설\n\n35회 ‘국민’ 언급하며 감사 인사\n남북 정상...,[고용의질이 좋아졌다고...? 망상에빠졌니.. 아니면.. 문꼬리몇인방들이.. 사실을...
4329,https://n.news.naver.com/mnews/article/088/000...,"文대통령 ""코로나 위기 이전 수준 가장 빨리 회복""",매일신문. All rights reserved. 무단 전재 및 재배포 금지.,"野 ""고장난 라디오처럼 자화자찬만""\n경제·문화 분야 성과 설명에 치중…민주당 의우...",[저 SGG 면상 좀 치워라 재수없다. 하루 빨리 깜방에 들어가는 모습이나 봤으면 ...


In [9]:
# 전처리 전 데이터 저장
df_moon.to_csv('news_data_문.csv', encoding = 'utf-8')

In [51]:
data = pd.read_csv('news_data_문.csv', encoding = 'utf-8')

#### 전처리

In [52]:
%%time
print('전처리 전 학습용 샘플의 개수 :',len(data))

# document 열에서 중복인 내용이 있다면 중복 제거
data.drop_duplicates(subset = ['content'], inplace=True) 

# train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
# 공백은 empty 값으로 변경
data['content'] = data['content'].str.replace('^ +', "")

# document column이 비어있다면(공백), Null 값으로 변경
data['content'].replace('', np.nan, inplace=True)

# Null 값 제거
data = data.dropna(how='any')

#언론사만 나오도록 바꾸기
data['agency'] = data['agency'].str.split('.').str[0]

# index reset
data = data.reset_index(drop=True)
print('전처리 후 학습용 샘플의 개수 :',len(data))

전처리 전 학습용 샘플의 개수 : 4331
전처리 후 학습용 샘플의 개수 : 3406
CPU times: total: 46.9 ms
Wall time: 56.4 ms


In [53]:
# 최종 저장
data.to_csv('news_data_문_final.csv', encoding = 'utf-8')

### 박근혜 기사 수집

In [10]:
client_id = 'cO6CRJKb2WhCBFfWd0eL'
client_secret = 'rt1u6IgEaE'

base_url = 'https://openapi.naver.com/v1/search/news.json'
queries = [
    '"朴대통령" | "박 대통령" | "박근혜" +2013',
    '"朴대통령" | "박 대통령" | "박근혜" +2014',
    '"朴대통령" | "박 대통령" | "박근혜" +2015',
    '"朴대통령" | "박 대통령" | "박근혜" +2016',
    '"朴대통령" | "박 대통령" | "박근혜" +2017'
]
n_display = 100
sort = 'sim'

In [11]:
# 네이버 API 설정
all_links = []
all_pubdates = []

# HTML 태그 제거 함수
def remove_tag(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# 여러 해에 걸쳐 뉴스 링크와 발행일 가져오기
for query in queries:
    encQuery = urllib.parse.quote(query)
    for start in range(1, 1001, 100):
        url = f'{base_url}?query={encQuery}&display={n_display}&start={start}&sort={sort}'
        my_request = urllib.request.Request(url)
        my_request.add_header("X-Naver-Client-Id", client_id)
        my_request.add_header("X-Naver-Client-Secret", client_secret)
        
        try:
            response = urllib.request.urlopen(my_request)
            rescode = response.getcode()
            if rescode == 200:
                response_body = response.read()
            else:
                print("Error Code:" + rescode)
                continue
        except urllib.error.HTTPError as e:
            print(f"HTTP Error: {e.code} - {e.reason}")
            continue
        except urllib.error.URLError as e:
            print(f"URL Error: {e.reason}")
            continue

        search_results = response_body.decode('utf-8')
        search_results = json.loads(search_results)
        
        if 'items' not in search_results:
            print("No items in search results")
            continue

        p = re.compile('n.news.naver.com')
        for item in search_results['items']:
            if p.search(item['link']) is not None:
                title = html.unescape(remove_tag(item['title']))
                pubdate = item['pubDate']
                link = item['link'].replace('\\', '')
                all_links.append(link)
                all_pubdates.append(pubdate)
        
        time.sleep(1)  # API 호출 제한 방지를 위한 대기 시간

df_link = pd.DataFrame({'link': all_links, 'pubdate': all_pubdates})

In [12]:
df_link

,link,pubdate
0,https://n.news.naver.com/mnews/article/421/000...,"Thu, 30 Jun 2022 16:03:00 +0900"
1,https://n.news.naver.com/mnews/article/003/001...,"Tue, 10 Nov 2020 12:49:00 +0900"
2,https://n.news.naver.com/mnews/article/469/000...,"Tue, 10 Nov 2020 20:21:00 +0900"
3,https://n.news.naver.com/mnews/article/032/000...,"Tue, 10 Nov 2020 16:25:00 +0900"
4,https://n.news.naver.com/mnews/article/421/000...,"Tue, 10 Nov 2020 20:55:00 +0900"
...,...,...
3762,https://n.news.naver.com/mnews/article/144/000...,"Tue, 25 Oct 2016 10:11:00 +0900"
3763,https://n.news.naver.com/mnews/article/003/000...,"Tue, 25 Oct 2016 08:31:00 +0900"
3764,https://n.news.naver.com/mnews/article/079/000...,"Tue, 25 Oct 2016 05:02:00 +0900"
3765,https://n.news.naver.com/mnews/article/081/000...,"Tue, 25 Oct 2016 03:37:00 +0900"


In [13]:
# Selenium을 사용하여 각 링크에서 데이터 수집
driver = webdriver.Chrome()
driver.implicitly_wait(3)

In [14]:
results = []

for i, url in enumerate(df_link['link']):
    driver.get(url)
    
    # 제목, 본문, 언론사 얻기
    try:
        title = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="title_area"]/span'))
        ).text
    except Exception as e:
        print(f"Error fetching title for URL {url}: {e}")
        title = ""

    try:
        content = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="newsct_article"]'))
        ).text
    except Exception as e:
        print(f"Error fetching content for URL {url}: {e}")
        content = ""

    try:
        copyright_text = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.XPATH, '//p[@class="c_text"]'))
        ).text
        if "ⓒ" in copyright_text:
            agency = copyright_text.split("ⓒ")[1].strip()
        else:
            agency = ""
    except Exception as e:
        print(f"Error fetching agency for URL {url}: {e}")
        agency = ""
    
    # 댓글 얻기
    try:
        all_comment_btn = WebDriverWait(driver, 1).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'u_cbox_in_view_comment'))
        )
        all_comment_btn.click()
        
        comments = []
        while True:
            try:
                more_comment_btn = WebDriverWait(driver, 1).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, 'u_cbox_btn_more'))
                )
                more_comment_btn.click()
                time.sleep(1)
            except:
                break
        
        comments_elements = driver.find_elements(By.CLASS_NAME, 'u_cbox_contents')
        comments = [x.text for x in comments_elements]
    except Exception as e:
        print(f"Error fetching comments for URL {url}: {e}")
        comments = []

    time.sleep(1)
    results.append([url, title, agency, content, comments])

driver.quit()

Error fetching comments for URL https://n.news.naver.com/mnews/article/057/0000778498?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/014/0003766530?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/018/0003731252?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/014/0003507285?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/015/0003690673?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/009/0003830501?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/008/0003121348?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/057/0000986450?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/081/0002719263?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/001/0008006173?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/057/0000835815?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/057/0000835750?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/021/0002257451?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/008/0003579163?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/008/0003576766?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/018/0003391088?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/008/0003572952?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/009/0003595620?sid=101: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/016/0001250035?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/037/0000023218?sid=004: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/030/0002591482?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/030/0002591104?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0002603237?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/030/0002591096?sid=105: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0002602711?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/005/0000976460?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0002601537?sid=102: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/016/0001182334?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/081/0002788874?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/014/0003758019?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/081/0002787704?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/014/0003757352?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/081/0002787279?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/018/0003716617?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/020/0003031796?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/022/0003131412?sid=102: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/081/0002786946?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/014/0003752970?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/382/0000529313?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/057/0001062744?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/057/0001062636?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/057/0001059795?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/014/0003746424?sid=102: Message: 

Error fetching title for URL https://n.news.naver.com/mnews/article/057/0001058007?sid=102: Message: 

Error fetching content for URL https://n.news.naver.com/mnews/article/057/0001058007?sid=102: Message: 

Error fetching agency for URL https://n.news.naver

...

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0001049889?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0001009512?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0000779027?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/014/0002996695?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/020/0002469503?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/022/0002529936?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/082/0000387382?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/037/0000023218?sid=004: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/011/0002990746?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/127/0000023949?sid=004: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/018/0003692507?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0002428776?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0002427535?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0002420652?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0002420289?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/020/0003022371?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/081/0002777208?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/108/0002573116: Message: 

Error fetching comments for URL https://n.news.naver.c

Error fetching comments for URL https://n.news.naver.com/mnews/article/016/0001130480?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0007506691?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/022/0003103047?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/008/0003752171?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/005/0000941650?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/081/0002762314?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/277/0003841326?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/015/0003664060?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/031/0000388959?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/018/0003369510?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/020/0002866047?sid=101: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/020/0002866042?sid=101: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/020/0002838931?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/057/0000739503?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/057/0000739436?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0006679623?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/008/0003506919?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/003/0006542399?sid=100: Message: 

Error fetching comments for URL https://n.news

Error fetching comments for URL https://n.news.naver.com/mnews/article/081/0002788367?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/081/0002788144?sid=100: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0002492382?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/374/0000121292?sid=004: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/081/0002787769?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/081/0002787477?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0002488120?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0002487546?sid=102: Message: 

Error fetching comments for URL https://n.news.naver.com/mnews/article/421/0002487543?sid=102: Message: 

Error fetching comments for URL https://n.news

In [15]:
df_park = pd.DataFrame(results, columns=['url', 'title', 'agency', 'content', 'comment'])
df_park

,url,title,agency,content,comment
0,https://n.news.naver.com/mnews/article/421/000...,"'성상납 의혹' 기업 대표 조사…""이준석, 朴대통령 만나게 힘쓰겠다""(종합)","뉴스1. All rights reserved. 무단 전재 및 재배포, AI학습 이용...","""일정표, 카드 내역 등 자료 제출""\n김성진 아이카이스트 대표의 법률 대리인인 김...","[좌파들이 더 좋아하는 여당대표 이준석 ~~빨리 사라졌으면,, 사기죄로 복역중인 김..."
1,https://n.news.naver.com/mnews/article/003/001...,"與, 세월호 朴대통령 기록물 제출요구안 발의…범여권 참여",뉴시스. All rights reserved. 무단 전재 및 재배포 금지.,"고영인 의원 대표발의…與 130명 등 범여권 141명 동의\n""성역 없는 진상규명 ...",[또 밀린다싶으니 세월호 불쌍한 애들 시체팔이 하려는구나...하아... 민주당것들 ...
2,https://n.news.naver.com/mnews/article/469/000...,범여권 의원 141명 “세월호 朴대통령 기록물 제출” 요구안 발의,한국일보. All rights reserved. 무단 전재 및 재배포 금지.,세월호 참사 진상규명과 책임자 처벌을 요구하며 31일째 무기한 단식투쟁을 이어가는 ...,[당신보다 더 가슴아픈 사람들도 많다. 이용당하고 놀아나더니 우려먹고 그만하자.]
3,https://n.news.naver.com/mnews/article/032/000...,"민주당 등 의원 141명, 세월호 朴대통령 기록물 제출요구안 발의 “성역 없는 진상규명”",경향신문. All rights reserved. 무단 전재 및 재배포 금지.,"[경향신문]\n더불어민주당과 정의당, 열린민주당 등 국회의원 141명이 참여한 ‘세...","[세월호 팔이 지겹지도 않나. 먹을게 아직도 남아 있냐?, 문재인 대통령 덕분에 한..."
4,https://n.news.naver.com/mnews/article/421/000...,"범여 의원 141명, 세월호 朴대통령 기록물 제출 요구안 발의","뉴스1. All rights reserved. 무단 전재 및 재배포, AI학습 이용...","올해 내 처리 목표, 문제는 국민의힘 동참 여부\n민주, 국민의힘 지도부와 만나 설...","[울궈먹기 다시 시작인가..., 너들의 짓꺼리는 울겨먹고 또울겨먹고 또 그짓이냐또 ..."
...,...,...,...,...,...
3762,https://n.news.naver.com/mnews/article/144/000...,朴대통령 임기 내 개헌 가능 할까?,스포츠경향. All rights reserved. 무단 전재 및 재배포 금지.,박근혜 대통령이 24일 제346회 국회(정기회) 제10차 본회의에서 시정연설을 통해...,"[개헌이 필요할수도 있지 하지만 박근혜정부에선 아님, 아니 무슨 개헌이야 개헌을 박..."
3763,https://n.news.naver.com/mnews/article/003/000...,"'최순실, 朴대통령 연설문 사전열람' 보도에 野 경악…충격",뉴시스. All rights reserved. 무단 전재 및 재배포 금지.,【서울=뉴시스】박진희 기자 = 박근혜 대통령이 24일 국회에서 2017년도 예산안과...,[밤새 억울하고 가슴이 답답해서 잠이안와서 뜬눈으로 출근햇다. 말도안되는 일이 벌어...
3764,https://n.news.naver.com/mnews/article/079/000...,朴 대통령 개헌 노림수는 반(反)문재인 연대?,노컷뉴스. All rights reserved. 무단 전재 및 재배포 금지.,"與野 개헌론 ‘내각제’ 맞춰지면, 文 중심 ‘4년 중임제’ 고립\n\n[CBS노컷뉴...","[●⊙ 친일 비리왕국 = 쓰레기 똥누리당 ~!! ㅋㅋ, ""참 나쁜 대통령이다. 국민..."
3765,https://n.news.naver.com/mnews/article/081/000...,[朴대통령 “임기 내 개헌”] ‘국민’ 32회 최다 언급… ‘경제’ 28회·‘개헌’...,서울신문. All rights reserved. 무단 전재 및 재배포 금지.,[서울신문]\n24일 박근혜 대통령이 2017년 예산안 시정연설에서 가장 많이 사용...,"[어찌 이분은 조용히 있는게 도와주는것 같은데..., 최순실 묻으려구 물타기하네 ㅠ..."


In [16]:
# 전처리 전 데이터 저장
df_park.to_csv('news_data_박.csv', encoding = 'utf-8')

In [17]:
data = pd.read_csv('news_data_박.csv', encoding = 'utf-8')

In [18]:
%%time
print('전처리 전 학습용 샘플의 개수 :',len(data))

# document 열에서 중복인 내용이 있다면 중복 제거
data.drop_duplicates(subset = ['content'], inplace=True) 

# train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
# 공백은 empty 값으로 변경
data['content'] = data['content'].str.replace('^ +', "")

# document column이 비어있다면(공백), Null 값으로 변경
data['content'].replace('', np.nan, inplace=True)

# Null 값 제거
data = data.dropna(how='any')

#언론사만 나오도록 바꾸기
data['agency'] = data['agency'].str.split('.').str[0]

# index reset
data = data.reset_index(drop=True)
print('전처리 후 학습용 샘플의 개수 :',len(data))

전처리 전 학습용 샘플의 개수 : 3767
전처리 후 학습용 샘플의 개수 : 2970
CPU times: total: 0 ns
Wall time: 34.3 ms


In [19]:
# 최종 저장
data.to_csv('news_data_박_final.csv', encoding = 'utf-8')